IMPORTAMOS LAS LIBRERIAS NECESARIAS PARA NUESTRO CODIGO Y PODER TRANSFORMARLO A UNO MAS PROLIJO

In [1]:
import pandas as pd
import ast
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
ruta=r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Crudos\australian_user_reviews.json'
data=[]

with open(ruta,'r', encoding='utf-8') as file:
    for linea in file:
        try:
            json_datos = ast.literal_eval(linea)
            data.append(json_datos)
        except ValueError as e:
            print(f"Error en la línea: {linea}")
            continue

In [3]:
df_reviews = pd.DataFrame(data)

In [4]:
df_review_full=df_reviews.explode('reviews')

In [5]:
df_review_full=pd.concat([df_review_full.drop(['reviews'],axis= 1), df_review_full['reviews'].apply(pd.Series)], axis = 1)

In [6]:
df_review_full.rename(columns= {0: 'sentiment_analysis'}, inplace= True)

In [7]:
df_review_full.drop(['funny', 'posted', 'last_edited', 'helpful', 'user_url'], axis=1, inplace=True)

In [8]:
df_review_full['sentiment_analysis'] = 0

In [9]:
df_review_full.dropna(subset='item_id', inplace=True)

In [10]:
from textblob import TextBlob

In [11]:
def sentiment_score(review:str) -> int:
  
    if not review:
        return 1
    else:
        analisis = TextBlob(review)
        if analisis.sentiment.polarity < -0.2:
            return 0  
        elif analisis.sentiment.polarity > 0.2:
            return 2  
        else:
            return 1  

In [12]:
df_review_full['sentiment_analysis']=df_review_full['review'].apply(sentiment_score)

In [13]:
df_review_full.drop(columns='review', inplace= True)

In [15]:
df_review_full['recommend'].replace({True:1,False:0}, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_21524\533596738.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_review_full['recommend'].replace({True:1,False:0}, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_21524\533596738.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_review_

In [16]:
df_review_full.head()

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,1,1
0,76561197970982479,22200,1,2
0,76561197970982479,43110,1,1
1,js41637,251610,1,1
1,js41637,227300,1,1


In [17]:
df_review_full.to_parquet(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Limpios\reviews.parquet', compression='gzip')